In [5]:
%matplotlib inline

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats
import scipy.io.wavfile
from scipy.fftpack import fft, fftfreq

# Error-Correcting Codes

## What are Error-Correcting codes and why are they so important?

In the era of communications, it is always important the data to be transferred in a secure and looseless way. But we still cannot make the technologies so perfect, and sometime times part of the data can be lost during the transfer. 
So what can we do in order to check whether there are errors in data transmission over noisy or unreliable communications channels?
For example, we can generate a hashcode of the transferred data (using MD5, SHA256 or some other algorithm), and provide it together with the transferred data. If there is difference between the generated hashcode, and the hashcode, that we calculate at the final point of the transfer, we shall know, that there is difference between the original data, and the data that we received (some transfer loss, some intended change in the data being transferred).
If the content of the data is not so important for us, we can even just make a checksum - for example summing the numbers of the bytes to be transferred, and this sum will be transferred together with the data.  After the data is transferred, the checksum is calculated again, and if it is the same as the provided one, then we consider, that the transferred data is intact.
Ok, until now we found two easy ways to find that there was an error/problem during the data transmission. But what is the biggest problem with them?
The biggest problem is, that even if we find out, that a problem occurred during the data transfer, we can only detect it, but we CANNOT FIX it. And here come the error-correcting codes. 

## What are the different types of error-correcting codes? Provide real-world examples?

Linear Block Codes:

Hamming Codes: These are simple and widely used error-correcting codes that can correct single-bit errors and detect two-bit errors.
BCH Codes (Bose-Chaudhuri-Hocquenghem): These are a class of powerful error-correcting codes that can correct multiple random error patterns.
Reed-Solomon Codes: These codes are highly effective for correcting burst errors and are commonly used in digital storage and transmission systems, such as CDs, DVDs, and QR codes.
Convolutional Codes:

These codes are used in real-time error correction and are characterized by their use of convolutional processes to encode data streams. Convolutional codes are widely used in applications like mobile communications and satellite communications.
Viterbi Algorithm: Often used for decoding convolutional codes, this algorithm finds the most likely sequence of states that result in a given sequence of observed data.
Turbo Codes:

Turbo codes are a class of high-performance error-correcting codes that achieve near Shannon limit performance. They are used in deep-space communications and 4G/5G mobile networks.
These codes employ iterative decoding, which significantly improves error-correcting performance.
Low-Density Parity-Check (LDPC) Codes:

LDPC codes are linear block codes known for their excellent performance close to the Shannon limit. They use sparse matrices and iterative decoding techniques.
LDPC codes are used in modern communication standards, including Wi-Fi, 5G, and satellite communications.
Polar Codes:

Polar codes are a type of error-correcting code that can achieve the capacity of binary-input discrete memoryless channels. They are known for their simple structure and efficient decoding algorithms.
Polar codes have been adopted for use in 5G New Radio (NR) standard.
Product Codes:

Product codes are constructed by combining two or more simpler codes, usually block codes, to form a larger code with enhanced error-correcting capability.
They are used in applications where high reliability is required, such as data storage systems.
Repeat-Accumulate (RA) Codes:

These are a class of codes that combine simple repetition codes with accumulation (or differential encoding) to create more complex codes with good performance.
RA codes are used in applications where low complexity is essential, like in some wireless communication systems.
Each of these error-correcting codes has specific advantages and is chosen based on the requirements of the application, such as the nature of the errors (random or burst), the required error correction capability, the complexity of encoding/decoding, and latency constraints.


## What is a Hamming code? Describe the history and / or derive the formula(s).

## What are parity bits and how do we use them?

Let us first begin with this : what is Parity Check? The parity check is a simple error detection technique used to determine whether a binary data set has been altered during transmission or storage. For a parity check, we separate only one single bit, that the sender is responsible for tuning, and the rest bits are free to carry a message. The only job of this single bit is to make sure, that the total number of 1s in this binary data is an even number. For example, if the original binary data is [1101001], here the number of the 1s is 4, which is an even number. So the value of this special bit will be 0 and the transmitted data will be  [11010010] (the orinal data plus the special bit). But if we would like to transfer the data [1111 111], here the number of the 1s is odd, that is why the sender needs to flip that special bit to be 1, in order to make the count even and the transmitted data will be  [1111 1111]. And this special bit is named "parity bit". This is pretty simple, but still very elegant way a change anywhere in the binary data to be reflected in a single bit of information.
Of course, this is a very simple check - if there are two errors in the transmission, the number of the 1s still will be even, so the parity check will not show us, that there is a problem. Also if the parity check shows an error, it could be not one error, but 3 or 7 or 127. 
So parity checks on their own are pretty weak, but by distilling the idea of change across full message down to a single bit, give room for more sophisticated schemes.

## What is the Hamming distance and what is its significance? How is it related to other distance metrics for text / bit sequences?

## Deeper dive into mathematics:
Derive the general formula for the number of parity bits required for a given number of data bits.
Explain the process of encoding data using Hamming codes. How are parity bits positioned in the data?
Describe the process of detecting and correcting errors using Hamming codes. How are syndrome vectors used in this process?

Let's dive deeper into the mathematics:
To determine the number of parity bits required for a given number of data bits in a Hamming code, we need to ensure that the code can detect and correct single-bit errors. The process involves using the parity bits to cover all the data bits and the parity bits themselves. Here's the step-by-step derivation of the formula:
let us denote :
- d is the number of the data bits
- p is the number of the parity bits
- n is the total number of bits (n = d + p)
For hHamming code, each possible bit possition in the data bits must me uniquely identifiable by the combination of the parity bits. The parity bits can form 2 ** p combinations, and hence, we can conclude that these combinations should be greater that the total number of bits:

$$ 2^p >= n $$

Looks reasonable, correct? But we forget something - as it can happen, that there are errors in the data transmission, and we would like to be able to identify them uniquely, there is also the opportunity there not to be error at all, and one of the parity bits combinations should be used to indicate no error at all. That is why the condition should be:
$$ 2^p >= n + 1$$

And as 
$$ n = d + p $$
the formula for required parity bits for a given number of data bits in a Hamming code finally looks like this :
$$ 2^p >= d + p + 1 $$

Let us check how many parity bits for 10 data bits (d = 10). So we need to find such p, that 
$$ 2^p >= 10 + p + 1 $$
As $2^3$ is  8, obviously we shall need a bigger number. Let is try with $p = 4$. 
$$ 2^4 = 16 >= 10 + 4 + 1 $$
This is true, so obviously for 10 data bits 4 parity bits are sufficient.

So generally the method for finding the needed parity bits is :

Find the smallest p, that satisfies $ 2^p >= d + p + 1 $

### General algorithm for encoding data using the Hamming codes

The first step, that has to be done, is, of course, to determine the parity bits, that we need for our data.

The parity bits are placed at positions, which are powers of 2, starting from 0 - 1($2^0$), 2($2^1$), 4($2^2$),  8($2^3$) and so on. The data bits are placed in the remaining positions. 
But why it is chosen the parity bits to be at these places? The main idea is to choose the error-correcting bits such that the index-XOR (the XOR of all the bit positions containing a 1) is 0.(Source : https://en.wikipedia.org/wiki/Hamming_code). This choice allows each parity bit to cover a specific set of bit positions in a systematic way, ensuring that each data bit is checked by multiple parity bits in a non-overlapping manner. Each parity bit at position $2^k$ covers the overs all bit positions that have the k-th bit in their binary representation set to 1.
Each data bit is included in a unique set of 2 or more parity bits, as determined by the binary form of its bit position.

Thus :
- 1 Parity bit at position 1 covers all bit positions which have the least significant bit set: bit 1 (the parity bit itself), 3, 5, 7, 9, etc.
- 2 Parity bit at position 2 covers all bit positions which have the second least significant bit set: bits 2-3, 6-7, 10-11, etc.
- 3 Parity bit at position 4 covers all bit positions which have the third least significant bit set: bits 4-7, 12-15, 20-23, etc.
- 4 Parity bit at position 8 covers all bit positions which have the fourth least significant bit set: bits 8-15, 24-31, 40-47, etc.

## What are the limitations of Hamming codes? How do more advanced error-correcting codes address these limitations?

The Hamming codes are a fundamental class of of error correcting codes. They are effective for detecting and correcting single-bit errors in the transmitted data. But these codes have two main limitations : 
 - 1 Limited error correction and error detection capabilities - the Hamming codes can detect up to two bit errors and can correct one bit error.
 - 2 Redundancy overhead - the Hamming codes introduce additional parity bits. This in case of large data transmission can cause significant overhead.

As the Hamming codes were the "pioneers" among the error correcting codes, many other codes were developed after that. About 10 years later the Reed-Salomon code was developed. It has the possibility to detect and to correct a lot of errors. That is why the Reed-Salomon code is used for example for reading bar codes or QR codes, where the risk of damages is very high. Also Reed-Salomon codes are used in CDs and DVDs, where data can be subject to burst of errors.  
